## 1.1 Getting started

The following python packages are pre-installed, but listed here for your reference. This will install a handy set of command-line tools that will be useful when working with ADK.

In [ ]:
# !pip install -q google-adk>=1.12.0

In [ ]:
from helper import *

# Load environment variables from .env file
load_env()

## 1.2 Setting up your first agent

Before we dive into building agents, let's set up a new folder structure with ADK's built-in project scaffolding using the `adk create` command.

When you run `adk create`, it generates three essential files. 
1. The `.env` file securely stores your API credentials and configuration. 
2. The `__init__.py` file marks the directory as a Python package, nabling proper imports. 
3. Most importantly, the `agent.py` file provides a clean foundation where you'll implement your agent.

File structure:
```
app_01/
    __init__.py
    agent.py
    .env
```

ADK create supports two project types. The `--type=code` option generates a Python-based agent in `agent.py`. The `--type=config` option creates a YAML-based agent configuration.

The `--model` parameter specifies the LLM to be used by the agent. We will override this and experiment with different models as we create the agents in this lesson.

### Setting up authentication

Before using any AI models, the first step is to configure API credentials. There are several methods to authenticate with Google's Gemini models, including:
1. Google API key and 
2. Vertex AI based authentication.

For more information on setting up your own authentication, visit [Google](https://cloud.google.com/free?hl=en). 

The `--api_key` parameter automatically configures your project for Google's Gemini API, setting up the proper authentication environment variables. In this course your API key is configured for you already.

Run the cell below to create the folder structure for your agent!

In [ ]:
# First we create our expected agent folder 
# You can explore available option: !adk create --help 

!adk create --type=code app_01 --model gemini-2.0-flash-live-001 --api_key $GEMINI_API_KEY

## 1.3 Writing your first `agent.py`



In [ ]:
%%writefile app_01/agent.py

from google.adk.agents import Agent

root_agent = Agent(
    name="ai_news_agent_simple",
    model="gemini-2.0-flash-live-001", # Essential for live voice interaction
    instruction="You are an AI News Assistant.",
)

## 1.4 Running your first agent

Now comes the exciting part—actually using what you've built. ADK gives you several ways to run your agent, but for development and learning, you'll use the ADK Web UI.

To start the local server and access the Web UI, follow the below steps:

**Terminal Instructions**
1. To open the terminal, run the cell below.
2. Change to the current lesson directory with the following command: `cd L1`
4. Start your agent using the following command: `adk web --host 0.0.0.0 --port 8001`
5. Run the cell below the terminal, to display the proxy URL. (This step is only needed within this codelab. When running externally, you can directly access the local server link.)

Here's a closer look at the command that starts the agent.
- `adk web`: A simple command that starts your agent on the Web UI
- `--host 0.0.0.0`: The host address
- `--port 8001`: The port we will access the server from. In this case, port `8001`.

In [ ]:
# start a new terminal
import os
from IPython.display import IFrame

IFrame(f"{os.environ.get('DLAI_LOCAL_URL').format(port=8888)}terminals/1", 
       width=600, height=768)

In [ ]:
import os
print(os.environ.get('DLAI_LOCAL_URL').format(port='8001'))

## 1.5 Adding tools to your agent


In [ ]:
!adk create --type=code app_02 --model gemini-2.0-flash-live-001 --api_key $GEMINI_API_KEY

### Adding Google Search Tool


In [ ]:
%%writefile app_02/agent.py

from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="ai_news_agent_simple",
    model="gemini-2.0-flash-live-001", # Essential for live voice interaction
    instruction="You are an AI News Assistant. Use Google Search to find recent AI news.",
    tools=[google_search]
)


## 1.6 Adding a text model


In [ ]:
!adk create --type=code app_03 --model gemini-2.5-flash --api_key $GEMINI_API_KEY

Run the below cell to create a new agent. Notice how the only difference in the code is the "model" name.

In [ ]:
%%writefile app_03/agent.py

from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="ai_news_agent_simple",
    model="gemini-2.5-flash",
    instruction="You are an AI News Assistant. Use Google Search to find recent AI news.",
    tools=[google_search]
)


**Refresh the ADK Web UI**, and select the app_03 from the dropdown menu. Chat with your agent using text and ask it: "Give me the latest AI news".

*Bonus:* Explore the different capabilities in the Web UI like the Events tab.

## 1.7 Alternative development approaches

ADK supports a declarative approach using YAML files. This is ideal for when you want rapid prototyping without code or want configuration or declarative style development.

Create a new folder for the agent by running the cell below. Notice that this time you use `--type=config`. This will create a new directory for your agent. 

```
app_04/
    __init__.py
    root_agent.yaml
    .env
```

A new file for your agent called `root_agent.yaml` is created and you will use that to configure your agent.

In [ ]:
# You can also define the agent with simple YAML confirguration 

!adk create --type=config app_04 --model gemini-2.5-flash --api_key $GEMINI_API_KEY

### Building agents with the YAML configuration

The config below maps directly to our Python version. You define the same core components—name, model, instructions, and tools—but in a declarative format. The schema validation ensures your configuration is correct.

Now, **refresh the ADK Web UI** and switch to the yaml version in the UI (select app_04 in the dropdown). Try interacting with the agent using voice and verify it is fully functional.

In [ ]:
%%writefile app_04/root_agent.yaml

name: ai_news_agent_yaml
model: gemini-2.0-flash-live-001 # Using a model that supports live voice
description: An agent that answers questions about recent news in Artificial Intelligence.
instruction: You are an AI News Assistant. Use Google Search to find and summarize recent AI news.
tools:
  - name: google_search


## 1.8 Fine-tuning agent instructions



In [ ]:
# You can also modify the instructions  

!adk create --type=code app_05 --model gemini-2.0-flash-live-001 --api_key $GEMINI_API_KEY

Write the updated agent code to the `agent.py` file.

In [ ]:
%%writefile app_05/agent.py

from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="ai_news_agent_strict",
    model="gemini-2.0-flash-live-001",
    instruction="""
    **Your Core Identity and Sole Purpose:**
    You are a specialized AI News Assistant. Your sole and exclusive purpose is to find and summarize recent news (from the last few weeks) about Artificial Intelligence.

    **Strict Refusal Mandate:**
    If a user asks about ANY topic that is not recent AI news, you MUST refuse.
    For off-topic requests, respond with the exact phrase: "Sorry, I can't answer anything about this. I am only supposed to answer about the latest AI news."

    **Required Workflow for Valid Requests:**
    1. You MUST use the `google_search` tool to find information.
    2. You MUST base your answer strictly on the search results.
    3. You MUST cite your sources.
    """,
    tools=[google_search]
)

# Resources

[Gemini API Models](https://ai.google.dev/gemini-api/docs/models#model-variations)